# Service 3.2.2 District Heating Network Optimization

## Imports

In [2]:
import logging
import pickle
import os
import glob
import threading
import sqlite3
import time

from typing import Tuple

import numpy as np
import pandas as pd
from io import StringIO

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from pymoo.problems.functional import FunctionalProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination

from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

## ML Model Class

In [5]:
class MLModel:

    """
    Class representing a machine learning model.

    Attributes:
        LHV (int): Lower Heating Value in kj/m3.
        LHV_ng (int): Lower Heating Value for natural gas.
        eta_lim (float): Limit for efficiency.
        zeros (int): Flag indicating whether to include zeros in the dataset.
        random_seed (int): Random seed for reproducibility.
        scaler (StandardScaler): Scaler object for data normalization.

    Methods:
        create_input(path: str, save_local_file: bool, **file_format: str) -> tuple:
            Create input data from files in the specified path and return processed dataframes.

        train_MLModel() -> tuple:
            Train the machine learning model and return the trained model, score, and scaler object.    
    """

    def __init__(self):

        self.LHV = 37411 # kj/m3
        self.LHV_ng = self.LHV # Used for conversion from m3/hr to 
        self.eta_lim = 1.3 
        self.zeros = 1
        self.random_seed = 1

        self.scaler = StandardScaler()

    # Creazione del dataframe


    def create_input(self, path: str, save_local_file: bool, **file_format:str): 

        """
        Create input data from files in the specified path and return processed dataframes.

        Args:
            path (str): The path to the files.
            save_local_file (bool): Flag indicating whether to save the processed dataset locally.
            **file_format (str): The file format to save the dataset in.

        Returns:
            tuple: A tuple containing three dataframes: df_30, dataset, and temp_df.
                - df_30: The processed dataframe with resampled data.
                - dataset: The filtered and transformed dataset.
                - temp_df: A temporary copy of the dataset before filtering.

        """

        #Genera una lista fatta da tutti i nomi che rientrano nella richiesta 

        nomifiles=(glob.glob(path))

        df=pd.DataFrame()

        for nomi in nomifiles:
            A0 = pd.read_csv(nomi, sep=';', header=None)
            df = pd.concat([df,A0])


        nomi_originali = df.iloc[:,2].unique() #Vediamo quante grandezze vengono studiate

        # elimina le colonne 'a' e 'b' dal dataframe
        df=df.drop(df.columns[0],axis=1)
        df=df.drop(df.columns[0],axis=1)


        df.columns = ['nome', 'orario', 'valore'] # Cambiamo il nome delle colonne

        df['valore'] = df['valore'].str.replace(',', '.') # Aggiustiamo i valori del dataframe 
        df['valore'] = df['valore'].str.strip() # Serve per togliere tutti gli spazi da quella colonna
        df['valore'] = df['valore'].astype(float) # Rendiamo la colonna dei numeri float


        # Crea un nuovo dataframe con gli orari come prima colonna

        df = df.pivot(index='orario', columns='nome', values='valore') #TODO questo pivot genera problemi

        df.index = pd.to_datetime(df.index)
        
        # Reimposta l'indice

        df_30 = df.resample('30T').mean()
        # df_30=df.resample('15T').interpolate()

        #Limitati al temo di funzionamento B1-2

        df_30 = df_30.loc['2023-06-07 00:00:00':'2023-10-17 19:00:00']

        dataset = df_30.copy()

        dataset['NG Consumption [kW]'] = dataset['CONSUMO GAS (30 minutos)'].diff()*(self.LHV/1800)

        # dataset['NG Consumption [kW]'] = dataset['NG Consumption [kW]'].shift(-1)

        dataset['eta'] = dataset['ENERGIA INSTANTANEA (15 minuto)']/(dataset['NG Consumption [kW]']+0.001)
        dataset['Boiler 1 Hours'] = dataset['Horas Funcionamiento Caldera 1 (15 minuto)'].diff()
        dataset['Boiler 2 Hours'] = dataset['Horas Funcionamiento Caldera 2 (15 minuto)'].diff()
        dataset['Boiler 3 Hours'] = dataset['Horas Funcionamiento Caldera 3 (15 minuto)'].diff()
        dataset['Boiler 3 Hours'] = dataset['Boiler 3 Hours'].replace(np.nan, 0)

        dataset['BH'] = dataset['Boiler 1 Hours'] + dataset['Boiler 2 Hours'] #TODO ci serve davvero?

        dataset=pd.DataFrame(dataset)

        if self.zeros==1:
            #Elimino gli zeri da boiler hours
            dataset['filter'] = dataset.apply(lambda row: 0 if row['eta'] < self.eta_lim and
                                            row['ENERGIA INSTANTANEA (15 minuto)'] > 50
                                            and row['NG Consumption [kW]'] > 50
                                            #and row['BH'] > 0.05
                                            else 1, axis=1) #applica il se
        else:
            # Filtro ma lascio gli zeri
            dataset['filter'] = dataset.apply(lambda row: 0 if row['eta'] < self.eta_lim else 1, axis=1) # applica il se

        # Elimino i nan

        dataset = dataset.loc[dataset['filter'] != 1]
        dataset = dataset.drop('BH', axis=1)
        temp_df = dataset.copy()
        dataset = dataset.drop('filter', axis=1)

        dataset.fillna(0, inplace=True)

        if (save_local_file == True and file_format == '.xlsx'):
            dataset.to_excel('resources/TrainingDataset.xlsx')
        elif (save_local_file == True and file_format == '.csv'):
            dataset.to_csv('resources/TraningDataset.csv')
        else:
            pass

        return df_30, dataset, temp_df
    
    def train_MLModel(self):

        """

        Train the machine learning model and return the trained model, score, and scaler object.

        Returns:
            tuple: A tuple containing the trained model, score, and scaler object.

        """

        dataset = self.create_input('resources/RVENA_23*.csv', save_local_file=False)[1]

        X = dataset.loc[:, ['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)']]  #Le x e y della mia F
        y = dataset.loc[:, ['eta']]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=self.random_seed)
        scaler_fitted = self.scaler.fit(X_train)
        X_train = scaler_fitted.transform(X_train)
        X_test = scaler_fitted.transform(X_test)

        # Definition of the ANN Model

        model = MLPRegressor(hidden_layer_sizes=(20, 100,300,100, 20),
                            max_iter=100000000,
                            verbose=True,
                            solver='adam',
                            learning_rate='adaptive',
                            random_state=self.random_seed,
                            activation='relu')
        
        trained_model = model.fit(X_train, y_train)

        pickle.dump(trained_model, open("resources/models/TrainedModel.pkl", 'wb'))

        # Valutazione delle prestazioni del modello sui dati di test

        score = trained_model.score(X_test, y_test)
        print(f'R^2 score: {score:.2f}')

        # Utilizzo del modello per fare previsioni sui dati di test

        X_pred = self.scaler.transform(X)    #Utilizzo lo stesso scaler che è stato fittato prima
        y_pred = model.predict(X_pred)

        return model, score, scaler_fitted

    # dataset = create_input('resources/RVENA_23*.csv', save_local_file=False)[1]
    # dataset.head()

## Optmizer Class

In [16]:
class Optimizer:

    def __init__(self, dataset, model, n_gen, pop_size):
        
        self.optimization_df = dataset.iloc[:48].copy()
        self.X = 1
        self.n = len(dataset)
        self.start_o = 0
        self.final_df = self.optimization_df[self.start_o:self.start_o + self.n]
        self.model = model
        self.fixed_value = 0.5
        self.ngen = n_gen
        self.pop_size = pop_size
        self.random_seed = 1

        self.scaler = StandardScaler()
      
    def f(self, x):

        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))

        X = self.optimization_df.loc[:,['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)']]  #Le x e y della mia F
        y = self.optimization_df.loc[:,['eta']]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=self.random_seed)
        scaler_fitted = self.scaler.fit(X_train)        
        
        x_matrix = np.hstack((self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values.reshape((self.n, 1)), x_matrix))
        x_matrix = pd.DataFrame(x_matrix, columns=['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)'])
       
        # Apply the scaler transformation to the decision variables matrix
        x_matrix_scaled = self.scaler.transform(x_matrix)
        
        # Calculate the sum of the model predictions for all timesteps
        eta=self.model.predict(x_matrix_scaled)
             
        f=self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values/eta
        
        return np.sum(f)
    
    def f_values(self, x):

        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))
        
        x_matrix = np.hstack((self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values.reshape((self.n, 1)), x_matrix))
        x_matrix = pd.DataFrame(x_matrix, columns=['ENERGIA INSTANTANEA (15 minuto)','TEMP IMP CALDERAS (15 minuto)'])

        
        # Apply the scaler transformation to the decision variables matrix
        x_matrix_scaled = self.scaler.transform(x_matrix)
        
        # Calculate the sum of the model predictions for all timesteps
        eta=self.model.predict(x_matrix_scaled)
        
        eta = np.clip(eta, a_min=None, a_max=1.3)

        f=self.final_df['ENERGIA INSTANTANEA (15 minuto)'].values/eta
    
        return f

    def g1(self, x):
        # Reshape the decision variables into a matrix with n rows and X columns
        x_matrix = x.reshape((self.n, self.X))
        
        # Calculate the constraint values for each timestep
        g = x_matrix[:, 2] - x_matrix[:, 1]
        g = np.max(g, axis=0)
        
        return g
    
    def optimize(self):
            
        self.termination = DefaultSingleObjectiveTermination(xtol=1e-800, cvtol=1e-600, ftol=0.05, period=200, n_max_gen=self.ngen, n_max_evals=1000000000)
        algorithm = NSGA2(pop_size=self.pop_size)
        self.best_objective_values = []  

        def callback(algorithm):

            print(f"Generation: {(100*algorithm.n_gen/self.ngen):.2f}%")
            best_objective_value = algorithm.pop.get("F").min()
            self.best_objective_values.append(best_objective_value)

        start = time.time()
        self.problem = FunctionalProblem(self.X * self.n, self.f, constr_ieq=[], xl=60, xu=90)
        res = minimize(self.problem, algorithm, self.termination, seed=1, callback = callback)
        self.temperature = res.X.reshape(self.n,self.X)
        end = time.time()

        df_solutions = pd.DataFrame(self.temperature, columns=['Temperatures'])
        final_df = self.optimization_df[['ENERGIA INSTANTANEA (15 minuto)', 'VOLUMEN INSTANTANEO (15 minuto)', 'NG Consumption [kW]',  'eta', 'Boiler 1 Hours', 'Boiler 2 Hours','Boiler 3 Hours']]
        final_df['Optmized Temperatures'] = df_solutions['Temperatures'].values
        
        self.gas_real = final_df['NG Consumption [kW]'].sum()/2 # Total NG Consumption in kWh
        self.optimized_gas = res.F/2 # Optmized Gas Consumptiopn in kWh
        

        solution = {"Solution":{
            "Total NG Consumption": f"{self.gas_real} kWh", 
            "Optmized Gas Consumption": f"{int(self.optimized_gas)} kWh", ##TODO no value returned
            "Total Gas Saved": f"{float((self.gas_real-self.optimized_gas))} kWh",
            "Saved Cost": f"{float(100*(1-self.optimized_gas/self.gas_real))}%", #TODO specify value
            "Strategy": final_df.to_json(orient='columns'),
            "Total Execution Time": f"{(end-start)/60} seconds"
        }}
        
        return solution, df_solutions

## Variables Instantiation

In [6]:
df_30 = MLModel().create_input(r'C:\Users\annatalini\OneDrive - Engineering Ingegneria Informatica S.p.A\DigiBUILD\DigiBUILD - Developement\s3_2_2_ENG\resources\requestData\RVENA_23*.csv', save_local_file=True)[2].iloc[:48]

In [7]:
df_30.head()

nome,CONSUMO GAS (30 minutos),ENERGIA ACUMULADA (30 minutos),ENERGIA INSTANTANEA (15 minuto),Horas Funcionamiento Caldera 1 (15 minuto),Horas Funcionamiento Caldera 2 (15 minuto),Horas Funcionamiento Caldera 3 (15 minuto),TEMP IMP CALDERA 1 (15 minuto),TEMP IMP CALDERA 2 (15 minuto),TEMP IMP CALDERA 3 (15 minuto),TEMP IMP CALDERAS (15 minuto),...,TEMPERATURA IMPULSION ANILLO (15 minuto),TEMPERATURA RETORNO ANILLO (15 minuto),VOLUMEN ACUMULADO (15 minuto),VOLUMEN INSTANTANEO (15 minuto),NG Consumption [kW],eta,Boiler 1 Hours,Boiler 2 Hours,Boiler 3 Hours,filter
orario,,,,,,,,,,,,,,,,,,,,,
2023-06-07 06:00:00,4518144.0,82838200.0,740.605,23398.18,23341.42,NaN,64.050,57.395,73.145,72.320,...,73.815,51.805,8215091.30,28.945,1517.223889,0.488131,0.0,0.0,0.0,0
2023-06-07 09:00:00,4518192.0,82838896.0,427.275,23398.18,23341.42,NaN,64.440,56.855,70.305,70.405,...,64.815,54.905,8215226.17,39.855,789.787778,0.540999,0.0,0.0,0.0,0
2023-06-07 14:00:00,4518257.0,82839800.0,954.240,23398.18,23341.42,NaN,63.275,55.940,68.245,68.360,...,62.665,50.775,8215630.78,56.220,1330.168889,0.717382,0.0,0.0,0.0,0
2023-06-07 17:30:00,4518322.0,82840704.0,968.480,23398.18,23341.42,NaN,61.850,55.030,68.760,68.635,...,63.445,51.500,8215765.65,50.765,1330.168889,0.728087,0.0,0.0,0.0,0
2023-06-07 20:30:00,4518376.0,82841504.0,1068.180,23398.18,23341.42,NaN,61.850,54.115,75.200,74.095,...,74.010,53.360,8215900.52,45.670,1101.546111,0.969709,0.0,0.0,0.0,0


In [17]:
import time
# start = time.time()
model = pickle.load(open(r'C:\Users\annatalini\OneDrive - Engineering Ingegneria Informatica S.p.A\DigiBUILD\DigiBUILD - Developement\s3_2_2_ENG\resources\models\TrainedModel.pkl', 'rb'))
optimizer = Optimizer(df_30, model, 100, 200).optimize()
solution = optimizer[0]
temps = optimizer[1]
# end = time.time()
# total = ((end - start)/60)
# print(f"total exectuion time: {total}")

Generation: 1.00%
Generation: 2.00%
Generation: 3.00%
Generation: 4.00%
Generation: 5.00%
Generation: 6.00%
Generation: 7.00%
Generation: 8.00%
Generation: 9.00%
Generation: 10.00%
Generation: 11.00%
Generation: 12.00%
Generation: 13.00%
Generation: 14.00%
Generation: 15.00%
Generation: 16.00%
Generation: 17.00%
Generation: 18.00%
Generation: 19.00%
Generation: 20.00%
Generation: 21.00%
Generation: 22.00%
Generation: 23.00%
Generation: 24.00%
Generation: 25.00%
Generation: 26.00%
Generation: 27.00%
Generation: 28.00%
Generation: 29.00%
Generation: 30.00%
Generation: 31.00%
Generation: 32.00%
Generation: 33.00%
Generation: 34.00%
Generation: 35.00%
Generation: 36.00%
Generation: 37.00%
Generation: 38.00%
Generation: 39.00%
Generation: 40.00%
Generation: 41.00%
Generation: 42.00%
Generation: 43.00%
Generation: 44.00%
Generation: 45.00%
Generation: 46.00%
Generation: 47.00%
Generation: 48.00%
Generation: 49.00%
Generation: 50.00%
Generation: 51.00%
Generation: 52.00%
Generation: 53.00%
Ge

C:\Users\annatalini\AppData\Local\Temp\ipykernel_20624\1981403846.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Optmized Temperatures'] = df_solutions['Temperatures'].values
C:\Users\annatalini\AppData\Local\Temp\ipykernel_20624\1981403846.py:101: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "Optmized Gas Consumption": f"{int(self.optimized_gas)} kWh", ##TODO no value returned
C:\Users\annatalini\AppData\Local\Temp\ipykernel_20624\1981403846.py:102: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in futur

In [18]:
print(optimizer)

({'Solution': {'Total NG Consumption': '26270.835555555554 kWh', 'Optmized Gas Consumption': '8435 kWh', 'Total Gas Saved': '17835.766342106912 kWh', 'Saved Cost': '67.89188834283249%', 'Strategy': '{"ENERGIA INSTANTANEA (15 minuto)":{"1686117600000":740.605,"1686128400000":427.275,"1686146400000":954.24,"1686159000000":968.48,"1686169800000":1068.18,"1686171600000":185.15,"1686204000000":626.67,"1686214800000":541.21,"1686232800000":1025.455,"1686245400000":968.485,"1686256200000":1068.185,"1686258000000":327.575,"1686290400000":867.42,"1686292200000":69.395,"1686301200000":543.585,"1686319200000":971.515,"1686331800000":925.255,"1686342600000":1168.13,"1686344400000":196.615,"1686376800000":682.375,"1686387600000":439.495,"1686405600000":983.08,"1686418200000":971.515,"1686429000000":1098.735,"1686430800000":196.615,"1686463200000":1480.405,"1686465000000":219.75,"1686477600000":612.98,"1686492000000":913.685,"1686504600000":959.95,"1686515400000":1110.305,"1686517200000":185.05,"168

In [22]:
df_30['Temperature Ottimizzate'] = temps['Temperatures'].values

## Final DF and Plots

In [23]:
final_df = df_30[['ENERGIA INSTANTANEA (15 minuto)', 'VOLUMEN INSTANTANEO (15 minuto)', 'NG Consumption [kW]',  'eta', 'Boiler 1 Hours', 'Boiler 2 Hours',
       'Boiler 3 Hours', 'Temperature Ottimizzate']]

In [24]:
final_df.head(5)

nome,ENERGIA INSTANTANEA (15 minuto),VOLUMEN INSTANTANEO (15 minuto),NG Consumption [kW],eta,Boiler 1 Hours,Boiler 2 Hours,Boiler 3 Hours,Temperature Ottimizzate
orario,,,,,,,,
2023-06-07 06:00:00,740.605,28.945,1517.223889,0.488131,0.0,0.0,0.0,89.867316
2023-06-07 09:00:00,427.275,39.855,789.787778,0.540999,0.0,0.0,0.0,89.853419
2023-06-07 14:00:00,954.240,56.220,1330.168889,0.717382,0.0,0.0,0.0,89.849354
2023-06-07 17:30:00,968.480,50.765,1330.168889,0.728087,0.0,0.0,0.0,89.937015
2023-06-07 20:30:00,1068.180,45.670,1101.546111,0.969709,0.0,0.0,0.0,89.994768
